# Feature engineering part 2 (Dealing with time)

In [1]:
import pandas as pd
from fep import FEP, load_features
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
# read data and split train and test
bids = pd.read_csv('../Data/facebook/bids.csv')
bidder = pd.read_csv('../Data/facebook/train.csv')
X_bidder = bidder.drop(['outcome'],axis=1)
y_bidder = bidder['outcome']
X_train, X_test, y_train, y_test = train_test_split(X_bidder, y_bidder, random_state=42, stratify=y_bidder)
df_train = pd.merge(bids, pd.concat([X_train, y_train], axis=1), on='bidder_id', how='inner')  
df_test = pd.merge(bids, pd.concat([X_test, y_test], axis=1), on='bidder_id', how='inner') 

# Encoding

unique_bidders = pd.concat([df_train['bidder_id'], df_test['bidder_id']]).unique()
bidder_mapping = {bidder_id: idx for idx, bidder_id in enumerate(unique_bidders)}
df_train['encoded_bidder_id'] = df_train['bidder_id'].map(bidder_mapping)
df_test['encoded_bidder_id'] = df_test['bidder_id'].map(bidder_mapping)

df_train.drop('bidder_id',axis=1,inplace=True)
df_test.drop('bidder_id',axis=1,inplace=True)

### Dealing with time

The idea is to analyze time-based behavior to identify features that help us to understand if a bidder is a bot.

The features can be categorized based on whether they describe a single bid or provide an aggregated view of a bidder's overall behavior.

Some ideas:
- Bid Timing Patterns: analyze timing between bids (e.g., intervals between bids).
- Activity Speed: Bots tend to act faster than humans.
- Round Duration: bots may place bids just before the auction ends
- Time of Day: bots can operate 24/7, so looking at the time of day when bids occur could reveal out-of-place activity
- Response to Other Bidder: If bids are placed immediately after another bidder, or if there's no sign of pause or delay in decision-making, this could be indicative of bot activity.
- Concurrent bid: Bot can place bids in two different auctions faster than humans.